# Word representation using TensorFlow-ELMo

## I. Text encoding

To use ELMo as a module, we have to use TensorFlow 1. 
Also, we have to deactivate eager execution in order to build a graph and execute it in a session.

In [1]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf

tf.disable_eager_execution()

# Load ELMo model from TensorFlow hun
elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=True)

elmo

In [2]:
# 
embeddings = elmo(tf.constant(["the cat is on the mat", "dogs are in the fog"]),
    signature="default",
    as_dict=True)['elmo']

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embeddings)
    print(message_embeddings)
    print(message_embeddings.shape)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


[[[ 3.0815458e-01  2.6630402e-01  2.3561311e-01 ... -3.7085709e-01
    1.6490501e-01 -7.2459459e-02]
  [ 5.1428783e-01 -1.3532351e-01  1.1090414e-01 ...  4.0468305e-02
   -4.7897547e-02  7.3659611e-01]
  [-2.5880380e-02 -7.2836205e-02 -7.9355925e-02 ... -2.9072428e-01
    7.2421390e-01  4.3863511e-01]
  [-3.4798026e-01 -2.9102378e-02 -8.1993032e-01 ... -9.2048419e-01
    2.1888085e-02  1.2106040e-01]
  [-2.1827453e-01 -1.3076580e-01 -2.5209627e-01 ... -2.9693589e-01
   -1.5828075e-01 -4.9007181e-02]
  [ 1.0072538e-01 -2.9534612e-02 -2.4494284e-01 ... -3.7235063e-01
   -1.4875787e-01  2.1592249e-01]]

 [[ 5.4578602e-02 -2.6427531e-01  4.6843794e-01 ... -1.4077097e-01
   -2.6568234e-01  4.5211998e-01]
  [ 8.0942981e-02  1.1583895e-01 -1.5670605e-01 ... -2.6896143e-01
    3.3871925e-01  1.1577199e-02]
  [-7.8964382e-02  9.4927502e-01 -6.1804920e-01 ... -6.3055885e-01
    3.0942994e-01  1.5378746e-01]
  [-6.7808926e-02  9.7187601e-02 -3.6225462e-01 ...  7.7500239e-02
   -6.1660964e-02  6.0

## II. Using ELMo

https://www.insofe.edu.in/insights/how-to-use-elmo-embedding-in-bidirectional-lstm-model-architecture/

In [3]:
sent_sent = tf.constant([#sentiment sentences
    'this is a good product',
    'this is a bad product',
    'i am happy with this',
    'i really cannot understant why is this product',
    'the product has many benefits',
    'i am not satisfied'
])

sent_labels = [1, 0, 1, 0, 1, 0]

sent_sent_test = tf.constant([#sentiment sentences
    'this is the best product', #1
    'this is the worst product', #0
    'i am not angry with this at all', #1
    'i really can hate this product', #0
    'the product has not many downs', #1
    'i am unsatisfied' #0
])

In [4]:
from tensorflow.keras.layers import Input, Lambda, Bidirectional, Dense, Dropout, LSTM
from tensorflow.keras.models import Model

def ELMoEmbedding(input_text):
    return elmo(tf.reshape(tf.cast(input_text, tf.string), [-1]), signature="default", as_dict=True)["elmo"]

def build_model():
    input_layer = Input(shape=(1,), dtype="string", name="Input_layer")
    embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ), name="Elmo_Embedding")(input_layer)
    BiLSTM = Bidirectional(LSTM(10, return_sequences= False, recurrent_dropout=0.2, dropout=0.2), name="BiLSTM")(embedding_layer)
    Dense_layer_1 = Dense(5, activation='relu')(BiLSTM)
    output_layer = Dense(1, activation='sigmoid')(Dense_layer_1)
    model = Model(inputs=[input_layer], outputs=output_layer, name="BiLSTM with ELMo Embeddings")
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

elmo_BiDirectional_model = build_model()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "BiLSTM with ELMo Embeddings"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     [(None, 1)]               0         
_________________________________________________________________
Elmo_Embedding (Lambda)      (None, None, 1024)        0         
_________________________________________________________________
BiLSTM (Bidirectional)       (None, 20)                82800     
_________________________________________________________________
dense (Dense)                (None, 5)                 105       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 82,911
Trainable params: 82,911
Non-trainable params: 0
_________________________________________________________________


In [11]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    sent_elmo_hisory = elmo_BiDirectional_model.fit(sent_sent, sent_labels, epochs=20, steps_per_epoch=6)
    train_prediction = elmo_BiDirectional_model.predict(sent_sent, steps=1)
    test_prediction = elmo_BiDirectional_model.predict(sent_sent_test, steps=1)
    elmo_embedding_after = session.run(embeddings)

Train on 6 samples
Epoch 1/20
6/6 [==============================] - 4s 489ms/step - batch: 2.5000 - size: 1.0000 - loss: 0.5696 - accuracy: 0.7222
Epoch 2/20
6/6 [==============================] - 3s 442ms/step - batch: 2.5000 - size: 1.0000 - loss: 0.4097 - accuracy: 0.8889
Epoch 3/20
6/6 [==============================] - 3s 440ms/step - batch: 2.5000 - size: 1.0000 - loss: 0.2839 - accuracy: 1.0000
Epoch 4/20
6/6 [==============================] - 3s 440ms/step - batch: 2.5000 - size: 1.0000 - loss: 0.1845 - accuracy: 1.0000
Epoch 5/20
6/6 [==============================] - 3s 440ms/step - batch: 2.5000 - size: 1.0000 - loss: 0.1317 - accuracy: 1.0000
Epoch 6/20
6/6 [==============================] - 3s 437ms/step - batch: 2.5000 - size: 1.0000 - loss: 0.0967 - accuracy: 1.0000
Epoch 7/20
6/6 [==============================] - 3s 443ms/step - batch: 2.5000 - size: 1.0000 - loss: 0.0763 - accuracy: 1.0000
Epoch 8/20
6/6 [==============================] - 3s 490ms/step - batch: 2.500

In [12]:
train_prediction

array([[0.9861125 ],
       [0.00486219],
       [0.93559265],
       [0.00260982],
       [0.98661697],
       [0.00405416]], dtype=float32)

In [13]:
test_prediction

array([[0.98496413],
       [0.18841314],
       [0.23608506],
       [0.00625518],
       [0.9826081 ],
       [0.03110862]], dtype=float32)

In [14]:
print(elmo_embedding_after)

[[[ 3.0815458e-01  2.6630402e-01  2.3561311e-01 ... -3.7085709e-01
    1.6490501e-01 -7.2459459e-02]
  [ 5.1428783e-01 -1.3532351e-01  1.1090414e-01 ...  4.0468305e-02
   -4.7897547e-02  7.3659611e-01]
  [-2.5880380e-02 -7.2836205e-02 -7.9355925e-02 ... -2.9072428e-01
    7.2421390e-01  4.3863511e-01]
  [-3.4798026e-01 -2.9102378e-02 -8.1993032e-01 ... -9.2048419e-01
    2.1888085e-02  1.2106040e-01]
  [-2.1827453e-01 -1.3076580e-01 -2.5209627e-01 ... -2.9693589e-01
   -1.5828075e-01 -4.9007181e-02]
  [ 1.0072538e-01 -2.9534612e-02 -2.4494284e-01 ... -3.7235063e-01
   -1.4875787e-01  2.1592249e-01]]

 [[ 5.4578602e-02 -2.6427531e-01  4.6843794e-01 ... -1.4077097e-01
   -2.6568234e-01  4.5211998e-01]
  [ 8.0942981e-02  1.1583895e-01 -1.5670605e-01 ... -2.6896143e-01
    3.3871925e-01  1.1577199e-02]
  [-7.8964382e-02  9.4927502e-01 -6.1804920e-01 ... -6.3055885e-01
    3.0942994e-01  1.5378746e-01]
  [-6.7808926e-02  9.7187601e-02 -3.6225462e-01 ...  7.7500239e-02
   -6.1660964e-02  6.0